In [ ]:
from IPython.core.debugger import set_trace
import numpy as np
import pandas as pd
import itertools
import glob
from openpyxl import load_workbook
Auftrag = './absys_files/A-2017-0059_919A.XLSX'

In [ ]:
def findZählernummer(ZNr, df):
    if isinstance(int(ZNr), int):
        assert len(str(ZNr)) == 8
        return df[(df['ZählernummerAlt'] == ZNr)]

In [ ]:
def protoCleanupHeizung(df):
    df2append = df.dropna()
    df2append = df2append[(df2append['Baulänge'] != 'Baulänge') &
                          (df2append['Bauhöhe'] != 'Bauhöhe')&
                          (df2append['Bautiefe'] != 'Bautiefe') &
                          (df2append['ZählernummerAlt'] != 'Altgerät') &
                          (df2append['ZählernummerNeu'] != 'Neugerät')]
    df2append = df2append[(df2append['Baulänge'] != 'D') &
                          (df2append['Bauhöhe'] != 'E') &
                          (df2append['Bautiefe'] != 'F') &
                          (df2append['ZählernummerAlt'] != 'Nr.') &
                          (df2append['ZählernummerNeu'] != 'Nr.')]
    return df2append

def protoCleanupWasser(df):
    df2append = df.dropna()
    df2append = df2append[(df2append['ZählernummerAlt'] != 'Gerätenummer') &
                          (df2append['ZählerStandAlt'] != 'Stand in m³')&
                          (df2append['ZählernummerNeu'] != 'Gerätenummer') &
                          (df2append['ZählerStandNeu'] != 'Stand in m³')]
    df2append = df2append[(df2append['ZählernummerAlt'] != 'E') &
                          (df2append['ZählerStandAlt'] != 'J') &
                          (df2append['ZählernummerNeu'] != 'Nr.') &
                          (df2append['ZählerStandNeu'] != 'Mont. Pkt.')]
    df2append = df2append[(df2append['ZählerStandAlt'] != 'Bowa')]
    df2append = df2append[(df2append['ZählerStandAlt'] != 'Veha')]
    df2append = df2append[(df2append['ZählerStandAlt'] != 'Fix')]
    df2append = df2append[(df2append['ZählerStandAlt'] != 'Kermi')]
    return df2append

In [ ]:
def collectProtocols(pathPattern='./absys_files/SenftenbergerRing*'):
    
    #Hier werden die Heizkostenverteilerdaten eingelesen
    protoColumns = {'Baulänge':str,
                    'Bauhöhe':str,
                    'Bautiefe':str,
                    'ZählernummerAlt':str,
                    'ZählerStandAlt':str,
                    'ZählernummerNeu':str}
    protokolle = glob.glob(pathPattern)
    #ToDo: Fehlerbehandlung wenn keine Dateien gefunden werden.
    protoDF = pd.DataFrame(columns=protoColumns.keys())
    for protokol in protokolle:
        protoWB = load_workbook(protokol)
        #Es wird vorrausgesetzt, dass das Worksheet "Dokumentation" heißt.
        try:
            protoWS = protoWB['Dokumentation']
        except:
            print("In der Datei "+protokol+" wurde kein Worksheet 'Dokumentation' gefunden!")
            break
        df = pd.read_excel(protokol, 
                           sheetname=0, 
                           names=protoColumns.keys(), 
                           dtype=protoColumns,  
                           parse_cols="E:G,L,R,V")
        df2appendHeizung = protoCleanupHeizung(df)
        pd.concat([protoDF, df2appendHeizung])
        set_trace()
    #Hier werden die Heizkostenverteilerdaten eingelesen
    protoColumns = {'ZählernummerAlt':str,
                    'ZählerStandAlt':str,
                    'ZählernummerNeu':str,
                    'ZählerStandNeu':str,}
    protokolle = glob.glob(pathPattern)
    #ToDo: Fehlerbehandlung wenn keine Dateien gefunden werden.
    protoDF = pd.DataFrame(columns=protoColumns.keys())
    for protokol in protokolle:
        protoWB = load_workbook(protokol)
        #Es wird vorrausgesetzt, dass das Worksheet "Dokumentation" heißt.
        try:
            protoWS = protoWB['Dokumentation']
        except:
            print("In der Datei "+protokol+" wurde kein Worksheet 'Dokumentation' gefunden!")
            break
        df = pd.read_excel(protokol, 
                           sheetname=0, 
                           names=protoColumns.keys(), 
                           dtype=protoColumns,  
                           parse_cols="F,K,L,W")
        df2appendWasser = protoCleanupWasser(df)
        pd.concat([protoDF, df2appendWasser])
        set_trace()
    protoDF.reset_index(drop=True, inplace=True)
    return protoDF

In [ ]:
#Nimmt an, dass das Excel-sheet IMMER 'Zählerliste' heißt. Tippfehler gehen schief!!!
df = pd.read_excel(Auftrag, sheetname='Zählerliste', header=9)
protoDF = collectProtocols()
#Es wird vorrausgesetzt, das die Zaehlernummern immer in der 5.(E) Spalte stehen.
for Zählernummer in df['alte\nZähler-Nr']:
    #Fehlerbehandlung, wenn Feldinhalt keine Zählernummer
    try:
        proto = findZählernummer(Zählernummer, protoDF)
        if (len(proto['ZählernummerNeu'].values) > 0):
            #print('Zählernummer '+str(Zählernummer)+' gefunden')
            df.loc[(df['alte\nZähler-Nr'] == Zählernummer),'neue\nZähler-Nr'] = proto['ZählernummerNeu'].values[0]
            #ToDo: Fehlerbehandlung wenn ein gültiger Zählerstand vorhanden.
            df.loc[(df['alte\nZähler-Nr'] == Zählernummer),'Stand\nAusbau'] = proto['ZählerStandAlt'].values[0]
        else:
            print('Zählernummer '+str(Zählernummer)+' nicht gefunden')
    except:
        print('Bei der Bearbeitung von Zählernummer '+str(Zählernummer)+' ist ein Fehler aufgetreten')

In [ ]:
protoDF